## Data Extraction with Docling

In this notebook, we'll extract content from PDFs into structured formats:

- **Markdown**: Full document text with page breaks for chunking
- **Images**: Save pages containing large charts/diagrams (>500x500 pixels)
- **Tables**: Extract with 2 paragraphs of context + page number metadata

**Pipeline Overview:**
1. **This Notebook (06-01)**: Extract PDFs → Markdown, Images, Tables
2. **Next Notebook (06-02)**: Load into vector database with embeddings
3. **Notebook 07**: Intelligent search with filters and reranking

**Output Structure:**
```
data/markdown/{company}/{document}.md
data/images/{company}/{document}/page_5.png
data/tables/{company}/{document}/table_1_page_5.md
```

### 1. Setup and Configuration

In [1]:
from pathlib import Path
from typing import List, Tuple

from docling_core.types.doc import PictureItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

In [2]:
# Directory paths
DATA_DIR = "data/rag-data/pdfs"
OUTPUT_MD_DIR = "data/rag-data/markdown"
OUTPUT_IMAGES_DIR = "data/rag-data/images"
OUTPUT_TABLES_DIR = "data/rag-data/tables"

### 2. Helper Functions

In [3]:
def extract_metadata_from_filename(filename: str) -> dict:
    """
    Extract metadata from filename.
    
    Expected format: CompanyName DocType [Quarter] Year.pdf
    Examples:
        - Amazon 10-Q Q1 2024.pdf
        - Microsoft 10-K 2023.pdf
    """
    parts = filename.split()
    
    return {
        'company_name': parts[0],
        'doc_type': parts[1],
        'fiscal_quarter': parts[2] if len(parts) == 4 else None,
        'fiscal_year': int(parts[-1])
    }

In [4]:
def extract_context_and_table(lines: List[str], table_index: int) -> Tuple[str, int]:
    """
    Extract context and table content at a specific position.
    
    Args:
        lines: All markdown lines
        table_index: Where the table starts
    
    Returns:
        (combined_content, next_line_index)
    """
    # Get 2 paragraphs before the table
    context_lines = []
    para_count = 0
    j = table_index - 1
    
    while j >= 0 and para_count < 2:
        if lines[j].strip():
            if '<!-- page break -->' not in lines[j]:
                context_lines.insert(0, lines[j])
        elif context_lines:
            para_count += 1
        j -= 1
    
    # Get all table rows
    table_lines = []
    i = table_index
    
    while i < len(lines):
        if lines[i].strip().startswith('|'):
            table_lines.append(lines[i])
            i += 1
        elif not lines[i].strip():
            i += 1  # Skip empty lines within table
        else:
            break  # End of table
    
    # Combine them
    content = '\n'.join(context_lines) + '\n\n' + '\n'.join(table_lines)
    return content, i

In [5]:
def extract_tables_with_context(markdown_text: str) -> List[Tuple[str, str, int]]:
    """
    Find all tables and extract them with context and page numbers.
    
    Returns:
        List of (content, table_name, page_number)
    """
    lines = markdown_text.split('\n')
    tables = []
    current_page = 1
    table_num = 1
    i = 0
    
    while i < len(lines):
        # Track page numbers
        if '<!-- page break -->' in lines[i]:
            current_page += 1
            i += 1
            continue
        
        # Found a table?
        if lines[i].strip().startswith('|') and lines[i].count('|') >= 2:
            content, next_i = extract_context_and_table(lines, i)
            tables.append((content, f"table_{table_num}", current_page))
            table_num += 1
            i = next_i
        else:
            i += 1
    
    return tables

In [6]:
def convert_pdf_to_docling(pdf_path: Path):
    """
    Convert PDF using Docling.
    
    Returns:
        Docling conversion result
    """
    pipeline_options = PdfPipelineOptions()
    pipeline_options.generate_picture_images = True
    pipeline_options.generate_page_images = True
    
    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
    )
    
    return converter.convert(pdf_path)

In [7]:
def save_page_images(result, images_dir: Path):
    """
    Find and save pages with large images (>500x500 pixels).
    """
    pages_to_save = set()
    
    for item in result.document.iterate_items():
        element = item[0]
        
        if isinstance(element, PictureItem):
            image = element.get_image(result.document)
            if image.size[0] > 500 and image.size[1] > 500:
                page_no = element.prov[0].page_no if element.prov else None
                if page_no:
                    pages_to_save.add(page_no)
    
    # Save images
    for page_no in pages_to_save:
        page = result.document.pages[page_no]
        page.image.pil_image.save(images_dir / f"page_{page_no}.png", "PNG")
    
    if pages_to_save:
        print(f"  ✓ Saved {len(pages_to_save)} page images")

In [8]:
def save_tables(markdown_text: str, tables_dir: Path):
    """
    Extract and save tables with context and page numbers.
    """
    tables = extract_tables_with_context(markdown_text)
    
    for table_content, table_name, page_num in tables:
        content_with_page = f"**Page:** {page_num}\n\n{table_content}"
        (tables_dir / f"{table_name}_page_{page_num}.md").write_text(content_with_page, encoding='utf-8')
    
    if tables:
        print(f"  ✓ Saved {len(tables)} tables")

### 3. Main Extraction Function

In [9]:
def extract_pdf_content(pdf_path: Path):
    """Extract PDF to markdown, images, and tables."""
    print(f"Processing: {pdf_path.name}")
    
    # Setup output directories
    filename = pdf_path.name.replace('.pdf', '')
    metadata = extract_metadata_from_filename(filename)
    company = metadata['company_name']
    filename_stem = pdf_path.stem
    
    md_dir = Path(OUTPUT_MD_DIR) / company
    images_dir = Path(OUTPUT_IMAGES_DIR) / company / filename_stem
    tables_dir = Path(OUTPUT_TABLES_DIR) / company / filename_stem
    
    for dir_path in [md_dir, images_dir, tables_dir]:
        dir_path.mkdir(parents=True, exist_ok=True)
    
    # Convert PDF with Docling
    result = convert_pdf_to_docling(pdf_path)
    
    # Save markdown
    markdown_text = result.document.export_to_markdown(page_break_placeholder="<!-- page break -->")
    (md_dir / f"{filename_stem}.md").write_text(markdown_text, encoding='utf-8')
    print(f"  ✓ Markdown saved")
    
    # Save page images
    save_page_images(result, images_dir)
    
    # Save tables
    save_tables(markdown_text, tables_dir)
    
    print(f"  Done!\n")

### 4. Process a Single PDF (Example)

In [10]:
# Find all PDF files
data_path = Path(DATA_DIR)
pdf_files = list(data_path.rglob("*.pdf"))
print(f"Found {len(pdf_files)} PDF files\n")

# Process one example first to see the output
if pdf_files:
    print("=== Processing Example PDF ===")
    extract_pdf_content(pdf_files[0])
    print("\nCheck the output folders to see extracted files!")
    print(f"- Markdown: {OUTPUT_MD_DIR}")
    print(f"- Images: {OUTPUT_IMAGES_DIR}")
    print(f"- Tables: {OUTPUT_TABLES_DIR}")

2025-12-13 18:38:56,615 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-13 18:38:56,642 - INFO - Going to convert document batch...
2025-12-13 18:38:56,643 - INFO - Initializing pipeline for StandardPdfPipeline with options hash afb4d61b52d512d984736b9faa45e3e9
2025-12-13 18:38:56,667 - INFO - Loading plugin 'docling_defaults'
2025-12-13 18:38:56,675 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-13 18:38:56,692 - INFO - Loading plugin 'docling_defaults'
2025-12-13 18:38:56,710 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']


Found 28 PDF files

=== Processing Example PDF ===
Processing: amazon 10-k 2023.pdf


2025-12-13 18:38:57,294 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-13 18:38:57,304 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-13 18:38:57,316 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-13 18:38:57,317 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-13 18:38:57,364 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-13 18:38:57,366 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-12-13 18:38:57,367 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-12-13 18:38:57,390 [RapidOCR] base.py:22: Using engine_name: onn

  ✓ Markdown saved
  ✓ Saved 61 tables
  Done!


Check the output folders to see extracted files!
- Markdown: data/rag-data/markdown
- Images: data/rag-data/images
- Tables: data/rag-data/tables


### 5. Process All PDFs

In [11]:
# Process all PDFs
print(f"\n=== Processing All {len(pdf_files)} PDFs ===\n")

for idx, pdf_path in enumerate(pdf_files, 1):
    print(f"[{idx}/{len(pdf_files)}]", end=" ")
    extract_pdf_content(pdf_path)

print("\n=== Extraction Complete ===")

2025-12-13 18:42:24,480 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-13 18:42:24,483 - INFO - Going to convert document batch...
2025-12-13 18:42:24,484 - INFO - Initializing pipeline for StandardPdfPipeline with options hash afb4d61b52d512d984736b9faa45e3e9
2025-12-13 18:42:24,484 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-13 18:42:24,497 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-13 18:42:24,501 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-13 18:42:24,502 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-13 18:42:24,550 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-13 18:42:24,552 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models


=== Processing All 28 PDFs ===

[1/28] Processing: amazon 10-k 2023.pdf


2025-12-13 18:42:24,801 - INFO - Auto OCR model selected rapidocr with onnxruntime.
2025-12-13 18:42:24,801 - INFO - Accelerator device: 'cuda:0'
2025-12-13 18:42:25,754 - INFO - Accelerator device: 'cuda:0'
2025-12-13 18:42:26,074 - INFO - Processing document amazon 10-k 2023.pdf
2025-12-13 18:43:03,759 - INFO - Finished converting document amazon 10-k 2023.pdf in 39.28 sec.
2025-12-13 18:43:03,918 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-13 18:43:03,921 - INFO - Going to convert document batch...
2025-12-13 18:43:03,922 - INFO - Initializing pipeline for StandardPdfPipeline with options hash afb4d61b52d512d984736b9faa45e3e9
2025-12-13 18:43:03,922 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-13 18:43:03,931 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-13 18:43:03,934 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-

  ✓ Markdown saved
  ✓ Saved 61 tables
  Done!

[2/28] Processing: amazon 10-k 2024.pdf


2025-12-13 18:43:04,155 - INFO - Auto OCR model selected rapidocr with onnxruntime.
2025-12-13 18:43:04,155 - INFO - Accelerator device: 'cuda:0'
2025-12-13 18:43:05,103 - INFO - Accelerator device: 'cuda:0'
2025-12-13 18:43:05,434 - INFO - Processing document amazon 10-k 2024.pdf
2025-12-13 18:43:39,433 - INFO - Finished converting document amazon 10-k 2024.pdf in 35.52 sec.
2025-12-13 18:43:39,645 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-13 18:43:39,649 - INFO - Going to convert document batch...
2025-12-13 18:43:39,650 - INFO - Initializing pipeline for StandardPdfPipeline with options hash afb4d61b52d512d984736b9faa45e3e9
2025-12-13 18:43:39,650 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-13 18:43:39,662 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-13 18:43:39,666 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-

...

  ✓ Markdown saved
  ✓ Saved 8 page images
  ✓ Saved 3 tables
  Done!

[27/28] Processing: meta 10-q q4 2024.pdf


[INFO] 2025-12-13 18:52:11,395 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-13 18:52:11,396 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-13 18:52:11,454 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-13 18:52:11,456 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-12-13 18:52:11,457 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-12-13 18:52:11,480 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-13 18:52:11,486 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-package

  ✓ Markdown saved
  ✓ Saved 9 page images
  ✓ Saved 3 tables
  Done!

[28/28] Processing: meta10-k 2023.pdf


2025-12-13 18:52:18,789 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-13 18:52:18,793 - INFO - Going to convert document batch...
2025-12-13 18:52:18,794 - INFO - Initializing pipeline for StandardPdfPipeline with options hash afb4d61b52d512d984736b9faa45e3e9
2025-12-13 18:52:18,794 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-13 18:52:18,803 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-13 18:52:18,807 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-13 18:52:18,808 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-13 18:52:18,851 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-13 18:52:18,853 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models

  ✓ Markdown saved
  ✓ Saved 67 tables
  Done!


=== Extraction Complete ===
